## **Application Programming Interface (API)**

En este notebook se planea trabajar con los datos procesados durante el ETL. Tengamos en cuenta que se va a separar en dos secciones:

1. Creacion de funciones segun el Henry Assesment:

    * def PlayTimeGenre( genero : str ), return **Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}**.

    * def UserForGenre( genero : str ), return: **Ejemplo de retorno: {"Usuario con más horas jugadas para genero" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}**.
    
    * def UsersRecommend( año : int ), return **Ejemplo de retorno: [{"Puesto 1" : genero_1}, {"Puesto 2" : genero_2},{"Puesto 3" : genero_3}]**

    * def UsersNotRecommend( año : int ), return **Ejemplo de retorno: [{"Puesto 1" : genero_1}, {"Puesto 2" : genero_2},{"Puesto 3" : genero_3}]**

    * def sentiment_analysis( año : int ), return **Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}**

2. Creacion de una API que nos permita interactuar con esta informacion de manera util a traves de una pagina web. Es necesario resaltar que los criterios de consumo de la API estaran basados segun API REST:

    * **Se Usara FastAPI**

Nuestro codigo se repartira en distintas secciones teniendo una similitud con el patron Modelo - Vista - Controlador (MVC):

* Celda de librerias.
* Celda de acciones.
* Celda de vistas.

## 1. Librerias

In [1]:
#Carga de librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#Carga de Datasets

steam_games_data = pd.read_parquet("data\steam_game_data")
user_items_data = pd.read_parquet("data\\user_items_data")
user_reviews_data = pd.read_parquet("data\\user_reviews_data")


## 2. Acciones

    Esta celda nos ayudara a crear todas las funciones que se vayan requiriendo durante el ejercicio.

## 2.1. Sub-Funciones

In [148]:
# Funcion que saca las columnas necesarias de un dataframe

def col_individuales(dataframe,lista_col):
    return pd.DataFrame(dataframe[lista_col])

def sentimientos(data):
    if data == 0:
        return "Negative"
    elif data == 1:
        return "Neutral"
    else:
        return "Positive"

    

## 2.2. Funciones "Main"

In [210]:
#* def PlayTimeGenre( genero : str )
# return **Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}**

#Variables Globales
genero_dicc= {}
def PlayTimeGenre(genero: str):

    try:
        if type(genero) == str:

            genero = genero.lower().title()
            playtime_data = pd.read_parquet("data\\playtime_data")
            lista_genero = playtime_data["genres"].unique()

            if genero in lista_genero:

                global genero_dicc
                data_genero = playtime_data[playtime_data["genres"] == genero]
                data_horas = data_genero.groupby("release_year")["playtime_forever"].sum()
                anio_max = data_horas.idxmax()

                anio_lanzamiento = f"Anio Lanzamiento con mas horas jugadas para {genero}"
                genero_dicc[anio_lanzamiento] = int(anio_max)
            
                return genero_dicc
            else:
                return print(f"La palabra {genero} no se encuentra en la base de datos")
        else:
            return print("Ingrese un valor valido: String")
    except NameError:
        return print("Ingrese un valor valido: String")

#def UserForGenre( genero : str ) 
#return: **Ejemplo de retorno: {"Usuario con más horas jugadas para genero" : us213ndjss09sdf, 
#"Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}**.

def UserForGenre(genero : str):
    try:
        if type(genero) == str:

            genero = genero.lower().title()
            usergenre_data = pd.read_parquet("data\\usergenre_data")
            lista_genero = usergenre_data["genres"].unique()

            if genero in lista_genero:

                data_genero = usergenre_data[usergenre_data["genres"] == genero]
                usuario_max = data_genero.groupby("user_id")["playtime_forever"].sum().idxmax()
                max_by_year = data_genero[data_genero["user_id"] == usuario_max].groupby("release_year")["playtime_forever"].sum()
                df_max_by_year = pd.DataFrame(max_by_year).reset_index()

                lista_aux = []

                for i in range(len(max_by_year)):

                    dicc_aux = {}
                    year = df_max_by_year["release_year"][i]
                    hour = df_max_by_year["playtime_forever"][i]
                    dicc_aux[year]=  hour
                    lista_aux.append(dicc_aux)
            
                return {"Usuario con mas horas jugadas para genero":usuario_max,
                    "Horas jugadas": lista_aux}
            else:
                return print(f"La palabra {genero} no se encuentra en la base de datos")
        else:
            return print("Ingrese un valor valido: String")
    except NameError:
        return print("Ingrese un valor valido: String")
    

#* def UsersRecommend( año : int ), 
# return **Ejemplo de retorno: [{"Puesto 1" : genero_1}, {"Puesto 2" : genero_2},{"Puesto 3" : genero_3}]**

def UsersRecommend(anio : int):
    try:
        if type(anio) == int:

            usersrecommend_data = pd.read_parquet("data\\usersrecommend_data")
            lista_anio = usersrecommend_data["release_year"].unique()

            if anio in lista_anio:
                
                data_bool = usersrecommend_data[usersrecommend_data["recommend"] == True]
                data_anio = data_bool[data_bool["release_year"] == anio]
                recommend_sum = data_anio.groupby("genres")["recommend"].sum()
                df_recommend_sum = pd.DataFrame(recommend_sum).sort_values("recommend",ascending=False).reset_index()
                
                list_aux = []

                for i in range(3):
                    dicc_aux = {}
                    dicc_aux[f"Puesto {i+1}"] = df_recommend_sum["genres"][i]
                    list_aux.append(dicc_aux)

                return list_aux

            else:
                return print(f"El anio {anio} no se encuentra en la base de datos")
        else:
            return print("Ingrese un valor valido: int")
    except NameError:
        return print("Ingrese un valor valido: int")
    
#* def UsersNotRecommend( año : int )
# return **Ejemplo de retorno: [{"Puesto 1" : genero_1}, {"Puesto 2" : genero_2},{"Puesto 3" : genero_3}]**

def UsersNotRecommend(anio : int):
    try:
        if type(anio) == int:

            usersrecommend_data = pd.read_parquet("data\\usersrecommend_data")
            lista_anio = usersrecommend_data["release_year"].unique()

            if anio in lista_anio:
                
                data_bool = usersrecommend_data[usersrecommend_data["recommend"] == False]
                data_anio = data_bool[data_bool["release_year"] == anio]
                recommend_sum = data_anio.groupby("genres")["recommend"].sum()
                df_recommend_sum = pd.DataFrame(recommend_sum).sort_values("recommend",ascending=False).reset_index()
                
                list_aux = []

                for i in range(3):
                    dicc_aux = {}
                    dicc_aux[f"Puesto {i+1}"] = df_recommend_sum["genres"][i]
                    list_aux.append(dicc_aux)

                return list_aux

            else:
                return print(f"El anio {anio} no se encuentra en la base de datos")
        else:
            return print("Ingrese un valor valido: int")
    except NameError:
        return print("Ingrese un valor valido: int")
    
#* def sentiment_analysis( año : int )
#  return **Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}**

def sentiment_analysis(anio : int):
    try:
        if type(anio) == int:

            sentiment_data = pd.read_parquet("data\\sentiment_data")
            lista_anio = sentiment_data["release_year"].unique()

            if anio in lista_anio:
                
                sentiment_data = pd.read_parquet("data\\sentiment_data").dropna()
                lista_anio = sentiment_data["release_year"].unique()
                data_anio = sentiment_data[sentiment_data["release_year"] == anio].drop(columns="item_id")
                df_data_anio = pd.DataFrame(data_anio.value_counts()).sort_index()

                list_aux = []
                list_sentiment=["Negative","Neutral","Positve"]


                for i, sentimiento in enumerate(list_sentiment):
                    dicc_aux = {}
                    dicc_aux[sentimiento] = int(df_data_anio.iloc[i])
                    list_aux.append(dicc_aux)

                list_aux

                return list_aux

            else:
                return print(f"El anio {anio} no se encuentra en la base de datos")
        else:
            return print("Ingrese un valor valido: int")
    except NameError:
        return print("Ingrese un valor valido: int")


## 3. Vistas

## 3.1 Data PlayTimeGenre

In [137]:
#Vamos a crear subdataframes para luego concatenarlos. Esto con el fin de ahorrar memoria y optimizar la informacion

#Data Steam Games
steam_sub = col_individuales(steam_games_data,["id","genres","release_year"])
steam_sub.rename(columns={'id':'item_id'},inplace=True) #Cambiamos el nombre en la columna id: item_id

#Data User Items
item_sub = col_individuales(user_items_data,["item_id","playtime_forever"])

#Unimos los dataframes

playtime_data = pd.merge(steam_sub,item_sub, on= "item_id", how="inner")

#Guardamos el dataset
playtime_data.to_parquet("data\\playtime_data", index=False)
print(f"El Dataset playtime_data.parquet ha sido creado")

El Dataset playtime_data.parquet ha sido creado


## 3.2 Data UserForGenre

In [8]:
#Vamos a crear subdataframes para luego concatenarlos. Esto con el fin de ahorrar memoria y optimizar la informacion

#Data Steam Games
steam_sub = col_individuales(steam_games_data,["id","genres","release_year"])
steam_sub.rename(columns={'id':'item_id'},inplace=True) #Cambiamos el nombre en la columna id: item_id

#Data User Items
item_sub = col_individuales(user_items_data,["user_id","item_id","playtime_forever"])

#Unimos los dataframes

usergenre_data = pd.merge(steam_sub,item_sub, on= "item_id", how="inner")

#Guardamos el dataset
usergenre_data.to_parquet("data\\usergenre_data", index=False)
print(f"El Dataset usergenre_data.parquet ha sido creado")


El Dataset usergenre_data.parquet ha sido creado


## 3.3 Data UsersRecommend

In [37]:
#Vamos a crear subdataframes para luego concatenarlos. Esto con el fin de ahorrar memoria y optimizar la informacion

#Data Steam Games
steam_sub = col_individuales(steam_games_data,["id","genres","release_year"])
steam_sub.rename(columns={'id':'item_id'},inplace=True) #Cambiamos el nombre en la columna id: item_id

#Data User Reviews
reviews_sub = col_individuales(user_reviews_data,["item_id","recommend"])

#Unimos los dataframes

usersrecommend_data = pd.merge(steam_sub,reviews_sub, on= "item_id", how="inner")

#Guardamos el dataset
usersrecommend_data.to_parquet("data\\usersrecommend_data", index=False)
print(f"El Dataset usersrecommend_data.parquet ha sido creado")

El Dataset usersrecommend_data.parquet ha sido creado


## 3.4 Data SentimentAnalysis

In [154]:
#Vamos a crear subdataframes para luego concatenarlos. Esto con el fin de ahorrar memoria y optimizar la informacion

#Data Steam Games
steam_sub = col_individuales(steam_games_data,["id","release_year"])
steam_sub.rename(columns={'id':'item_id'},inplace=True) #Cambiamos el nombre en la columna id: item_id

#Data User Reviews
reviews_sub = col_individuales(user_reviews_data,["item_id","sentiment"])

#Unimos los dataframes y limpiamos

sentiment_data = pd.merge(steam_sub,reviews_sub, on= "item_id", how="inner")
sentiment_data["feeling"] = sentiment_data["sentiment"].apply(sentimientos)
sentiment_data.drop(columns="sentiment", inplace=True)

#Guardamos el dataset
sentiment_data.to_parquet("data\\sentiment_data", index=False)
print(f"El Dataset sentiment_data.parquet ha sido creado")

El Dataset sentiment_data.parquet ha sido creado


## 4. Seccion de pruebas

In [212]:
PlayTimeGenre("Action")
PlayTimeGenre("Adventure")

print(f""" PlayTimeGenre: Succesfull 
      
      {genero_dicc} """)

#--------------------------------

print (f""" UserForGenre : Succesfull
       
       {UserForGenre("Adventure")}""")

print(f""" UsersRecommend: Succesfull
      
      {UsersRecommend(1997)}""")

print(f""" UsersNotRecommend: Succesfull
      
      {UsersNotRecommend(2012)}""")

[print(f""" sentiment_analysis: Succesfull
       
       {sentiment_analysis(1997)}""")]

 PlayTimeGenre: Succesfull 
      
      {'Anio Lanzamiento con mas horas jugadas para Action': 2012, 'Anio Lanzamiento con mas horas jugadas para Adventure': 2011} 
 UserForGenre : Succesfull
       
       {'Usuario con mas horas jugadas para genero': 'thugnificent', 'Horas jugadas': [{1987.0: 0.0}, {1988.0: 231.0}, {1989.0: 0.0}, {1990.0: 0.0}, {1991.0: 0.0}, {1992.0: 0.0}, {1993.0: 9578.0}, {1994.0: 544.0}, {1995.0: 0.0}, {1996.0: 2570.0}, {1997.0: 2512.0}, {1998.0: 0.0}, {1999.0: 0.0}, {2000.0: 48652.0}, {2001.0: 2430.0}, {2002.0: 128.0}, {2003.0: 880.0}, {2004.0: 0.0}, {2005.0: 2339.0}, {2006.0: 1391.0}, {2007.0: 725.0}, {2008.0: 2406.0}, {2009.0: 9226.0}, {2010.0: 3196.0}, {2011.0: 36955.0}, {2012.0: 89001.0}, {2013.0: 212088.0}, {2014.0: 186555.0}, {2015.0: 334257.0}, {2016.0: 109924.0}, {2017.0: 59573.0}, {2018.0: 0.0}]}
 UsersRecommend: Succesfull
      
      [{'Puesto 1': 'Adventure'}, {'Puesto 2': 'Action'}, {'Puesto 3': 'RPG'}]
 UsersNotRecommend: Succesfull
      
      

C:\Users\andre\AppData\Local\Temp\ipykernel_15864\1237915014.py:161: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  dicc_aux[sentimiento] = int(df_data_anio.iloc[i])


[None]